This notebook contains all the modeling steps after the feature selection one

In [26]:
!pip install scikit-optimize

In [0]:
%tensorflow_version 2.x
import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [0]:
import numpy as np
import joblib
import pandas as pd
import dill

from functools import partial

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.calibration import CalibratedClassifierCV
from sklearn.compose import make_column_transformer
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import StandardScaler, LabelBinarizer, LabelEncoder
from sklearn.metrics import log_loss, ndcg_score
from sklearn.utils import resample, check_X_y, check_array, check_random_state
from sklearn.utils.multiclass import type_of_target
from sklearn.utils._testing import SkipTest

from scipy.optimize import minimize

from skopt import forest_minimize, gp_minimize
from skopt.plots import plot_objective
from skopt.space import Real, Integer
from skopt.utils import use_named_args


from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import (
    ModelCheckpoint, EarlyStopping, TerminateOnNaN
)
from tensorflow.keras.layers import (
    Dense, Dropout, Activation, BatchNormalization, ELU
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2


import xgboost
from xgboost import XGBClassifier

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def preprocess_x(x_train, x_val):
  ohe_feats = ['gender', 'signup_method', 'signup_flow', 'language', 
              'affiliate_channel', 'affiliate_provider', 
              'first_affiliate_tracked', 'signup_app', 
              'first_device_type', 'first_browser']

  cat_strings = ohe_feats + ['age_interv', 'tfa_', 'dac_', 'booked', 'has_session']
  cat_features = [idx for idx, x in enumerate(x_train.columns) for string in cat_strings \
                  if string in x and 'action' not in x and 'epochs' not in x and 'msecs' not in x]
  
  num_features = [col for col in x_train.columns if col not in cat_features]
  
  ss = StandardScaler()
  preprocessor = make_column_transformer((ss, num_features), 
                                         remainder='passthrough')
  X_train = preprocessor.fit_transform(x_train)
  X_val = preprocessor.transform(x_val)
  return X_train, X_val


def plot_metrics(history):
  metrics =  ['loss', 'sparse_categorical_accuracy']
  plt.figure(figsize=(10, 10))
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch,  history.history[metric], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric], 
            linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    else:
      plt.ylim([0,1])
    plt.legend()

In [0]:
base_path = '/content/drive/My Drive/AML - AIRBNB/'
X_train_path = os.path.join(base_path, 'X_train_v2.pkl')
X_val_path = os.path.join(base_path, 'X_val.pkl')
y_train_path = os.path.join(base_path, 'y_train_v2.pkl')
y_val_path = os.path.join(base_path, 'y_val.pkl')

xgb_features_path = os.path.join(base_path, 'xgboost_features.pkl')

models_path = os.path.join(base_path, 'models')

In [0]:
X_train = pd.read_pickle(X_train_path)
X_val = pd.read_pickle(X_val_path)

y_train = pd.read_pickle(y_train_path)
y_val = pd.read_pickle(y_val_path)

In [0]:
xgb_features = pd.read_pickle(xgb_features_path)

In [0]:
xgb_features = X_train.columns.isin(xgb_features)

In [0]:
X_train, X_val = preprocess_x(X_train, X_val)

# MLP

---



In [0]:
def get_nn(X, y, neurons_1 = 256, neurons_2 = 128, neurons_3 = 64, l2_1 = 0.01,
           l2_2 = 0.01, lr=0.001):
  n_classes = len(np.unique(y))
  nn = Sequential()
  nn.add(Dense(neurons_1, input_dim=X.shape[1], kernel_regularizer=l2(l2_1)))
  nn.add(ELU())
  nn.add(BatchNormalization())
  nn.add(Dropout(0.5))    
  nn.add(Dense(neurons_2, kernel_regularizer=l2(l2_2)))
  nn.add(ELU())
  nn.add(BatchNormalization())
  nn.add(Dropout(0.4))    
  nn.add(Dense(neurons_3))
  nn.add(ELU())
  nn.add(BatchNormalization())
  nn.add(Dropout(0.4))   
  nn.add(Dense(n_classes))
  nn.add(Activation('softmax'))
  nn.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=lr),
              metrics=['sparse_categorical_accuracy',
                      'sparse_top_k_categorical_accuracy'])
  return nn

In [0]:
model = get_nn(X_train[:, xgb_features], y_train)

model.fit(X_train[:, xgb_features], y_train, epochs=500, batch_size=32, 
      validation_data=(X_val[:, xgb_features], y_val), verbose=2,
      callbacks=[EarlyStopping(patience=10, restore_best_weights=True)])

In [25]:
ndcg_score(LabelBinarizer().fit_transform(y_val), 
           model.predict_proba(X_val[:,xgb_features]))

0.8719705025340047

In [0]:
paramnames_nn = ['neurons_1', 'neurons_2', 'neurons_3', 'l2_1', 'l2_2', 'lr']

nn_space = [Integer(16, 1000, name='neurons_1'),
            Integer(16, 1000, name='neurons_2'),
            Integer(16, 1000, name='neurons_3'),
            Real(0.00001, 1, 'uniform', name='l2_1'), 
            Real(0.00001, 1, 'uniform', name='l2_2'),
            Real(0.0001, 0.1, 'uniform', name='lr')]

# XGBoost

---



In [0]:
LR_START = 0.1
LR_MIN = 0.008

In [0]:
def learning_rate_decay(boosting_round, num_boost_round, lr_start,
                        lr_min, lr_decay):
    learning_rate_start = lr_start
    learning_rate_min = lr_min
    lr_decay = lr_decay
    lr = learning_rate_start * np.power(lr_decay, boosting_round)
    return max(learning_rate_min, lr)


def get_xgb(num_class, lr_start=LR_START, lr_min=LR_MIN, lr_decay=0.98, 
            subsample_perc=1, lambda_reg=1):
  decaying_lr = partial(learning_rate_decay, lr_start=lr_start, lr_min=lr_min,
                        lr_decay=lr_decay)
  xgb = XGBClassifier(max_depth=7, n_estimators=1000, num_class=num_class,
                    objective='multi:softprob', gamma=0, min_child_weight=1,
                    max_delta_step=0, subsample=subsample_perc,
                    colsample_bytree=1,
                    missing=-1,
                    colsample_by_level=1,
                    reg_alpha=0, reg_lambda=lambda_reg, 
                    seed=42, tree_method = "gpu_hist",
                    callbacks=[xgboost.callback.reset_learning_rate(decaying_lr)]) 
  return xgb

In [56]:
xgb = get_xgb(12)
xgb.fit(X_train[:,xgb_features], y_train,
        eval_set=[(X_val[:, xgb_features], y_val)],
        eval_metric=['merror', 'mlogloss'],
        early_stopping_rounds=10, verbose=1)

[0]	validation_0-merror:0.360156	validation_0-mlogloss:2.14361
Multiple eval metrics have been passed: 'validation_0-mlogloss' will be used for early stopping.

Will train until validation_0-mlogloss hasn't improved in 10 rounds.
[1]	validation_0-merror:0.350261	validation_0-mlogloss:1.95117
[2]	validation_0-merror:0.344696	validation_0-mlogloss:1.81457
[3]	validation_0-merror:0.343815	validation_0-mlogloss:1.70737
[4]	validation_0-merror:0.341847	validation_0-mlogloss:1.62155
[5]	validation_0-merror:0.337781	validation_0-mlogloss:1.55067
[6]	validation_0-merror:0.336975	validation_0-mlogloss:1.48867
[7]	validation_0-merror:0.335026	validation_0-mlogloss:1.43783
[8]	validation_0-merror:0.332215	validation_0-mlogloss:1.39273
[9]	validation_0-merror:0.332196	validation_0-mlogloss:1.35298
[10]	validation_0-merror:0.33244	validation_0-mlogloss:1.31888
[11]	validation_0-merror:0.33304	validation_0-mlogloss:1.29029
[12]	validation_0-merror:0.331653	validation_0-mlogloss:1.26344
[13]	validati

XGBClassifier(base_score=0.5, booster='gbtree',
              callbacks=[<function reset_learning_rate.<locals>.callback at 0x7f046a2b3b70>],
              colsample_by_level=1, colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=7, min_child_weight=1, missing=-1, n_estimators=1000,
              n_jobs=1, nthread=None, num_class=12, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=42, silent=None, subsample=1, tree_method='gpu_hist',
              verbosity=1)

In [57]:
ndcg_score(LabelBinarizer().fit_transform(y_val), 
           xgb.predict_proba(X_val[:,xgb_features]))

0.8592073125985906

In [0]:
paramnames_xgb = ['lr_start', 'lr_min', 'lr_decay', 'subsample_perc', 'lambda']

xgb_space = [Real(0.05, 0.08, 'uniform', name='lr_start'),
             Real(0.0005, 0.05, 'uniform', name='lr_min'),
             Real(0.93, 0.99, 'uniform', name='lr_decay'),
             Real(0.6, 1, 'uniform', name='subsample_perc'), 
             Real(0.0001, 1, 'uniform', name='lambda_reg')]

# ENSEMBLING (WITH HPO INTEGRATION)

---



The EnsembleClasifier class is a sklearn-compatible estimator that implements the fitting of the base models (via the *__fit_nn* and *__fit_xgb* methods) as well as the fitting of the ensembler (via the *_fit_ensembler* method). The public *fit* method calss each of the aforementioned methods one by one. It also has a built-in HPO feature which can be activated through a boolean flag.

This class implements the first ensembler presented in the report.

In [0]:
class EnsembleClassifier(BaseEstimator, ClassifierMixin):
  def __init__(self, EPOCHS=500, BATCHSIZE=64, BOOTSTRAP_ITER=5, 
               load_models=False, save_models=False,
               random_state=42, hpo=False,):
    super(BaseEstimator, self).__init__()
    self.EPOCHS = EPOCHS
    self.BATCHSIZE = BATCHSIZE
    self.BOOTSTRAP_ITER = BOOTSTRAP_ITER
    self.load_models = load_models
    self.save_models = save_models
    self.random_state = random_state
    self.hpo = hpo
    self.__hpo_nn = use_named_args(nn_space)(self.__hpo_nn)
    self.__hpo_xgb = use_named_args(xgb_space)(self.__hpo_xgb)


  @staticmethod
  def _more_tags():
    return {'multioutput_only': True,
            'requires_fit': True}
  

  @staticmethod
  def objective_fun(w, X, y, n_class):
    predsize = len(w)
    w_range = np.arange(predsize) % n_class 
    for i in range(n_class): 
        w[w_range==i] = w[w_range==i] / np.sum(w[w_range==i])
        
    sol = np.zeros((X.shape[0], n_class))
    for idx in range(predsize):
        sol[:, idx % n_class] += X[:, idx] * w[idx]
    if np.isnan(sol).any():
      return np.inf
    ll = log_loss(y, sol)
    return ll


  def fit(self, X, y, **kwargs):
    X = check_array(X, dtype=[np.int_, np.intc, np.single, 
                              np.double, np.longdouble,
                              np.uint, np.int32, np.int64,
                              np.float32, np.float64])
    y = check_array(y, dtype = np.int_, ensure_2d=False)
    self.random_state_ = self.random_state
    self.X_train_ = X
    self.y_train_ = y.ravel()
    self.n_classes_ = len(np.unique(self.y_train_))
    self.classes_  = list(range(self.n_classes_))
    X_val = kwargs.get('X_val')
    y_val = kwargs.get('y_val')

    if X_val is not None and y_val is not None:
      self.X_val_ = X_val
      self.y_val_ = y_val
      self.val_data_ = (self.X_val_, self.y_val_)
      self.xgb_val_data_ = [self.val_data_]
      self.xgb_metrics_ = ['merror', 'mlogloss']
      self.xgb_es_ = 50
    else:
      self.X_val_ = None
      self.y_val_ = None
      self.val_data_ = None
      self.xgb_val_data_ = None
      self.xgb_metrics_ = None
      self.xgb_es_ = None
    
    nn_params = {}
    xgb_params = {}
    if self.hpo:
      print('ENTERING HPO...')
      xgb_params = self.__xgb_hpo_wrapper() 
      nn_params = self.__nn_hpo_wrapper()

    self.nn_models_ = self.__fit_nn(self.save_models, **nn_params)
    self.xgb_models_ = self.__fit_xgb(self.save_models, **xgb_params)

    print("FITTING ENSEMBLER...")
    self._fit_ensembler()
    return self

  def __fit_nn(self, save, **kwargs):
    nn_models_ = []
    for idx in range(self.BOOTSTRAP_ITER):
      term = TerminateOnNaN()
      self.nn_callbacks_ = [term]

      if self.X_val_ is not None and self.y_val_ is not None:
        nn_es = EarlyStopping(monitor='val_loss', patience=10, verbose=1, 
                                restore_best_weights=True) 
        self.nn_callbacks_.append(nn_es)

      self.random_state_ += 1
      random_state = check_random_state(self.random_state_)
      X, y = resample(self.X_train_, self.y_train_, random_state=random_state)
      nnpath = os.path.join(models_path, f'nn_{idx}.hdf5')
      
      if os.path.isfile(nnpath) and self.load_models:
        print(f"LOADING NN {idx}...\n\n")
        nn = load_model(nnpath)
      
      else:
        print(f"FITTING NN ON BOOTSTRAP {idx}\n\n")
        nn = get_nn(self.X_train_, self.y_train_, **kwargs)
        nn.fit(X, y, epochs=self.EPOCHS, batch_size=self.BATCHSIZE, 
              validation_data=self.val_data_, verbose=2,
              callbacks=self.nn_callbacks_)
        if save:
          nn.save(nnpath)
    
      nn_models_.append(nn)
    return nn_models_

  def __fit_xgb(self, save, **kwargs):
    xgb_models_ = []
    for idx in range(self.BOOTSTRAP_ITER):
      self.random_state_ += 1
      random_state = check_random_state(self.random_state_)
      X, y = resample(self.X_train_, self.y_train_, random_state=random_state)
      xgb = get_xgb(self.n_classes_, **kwargs)
      xgb_path = os.path.join(models_path, f'xgb_{idx}.pkl')

      if os.path.isfile(xgb_path) and self.load_models:
        print(f"LOADING XGB {idx}...\n\n")
        with open(xgb_path, 'rb') as f:
          xgb = dill.load(f)
      
      else:
        print(f"FITTING XGB ON BOOTSTRAP {idx}\n\n")
        xgb.fit(X, y, eval_set=self.xgb_val_data_,
                eval_metric=self.xgb_metrics_,
                early_stopping_rounds=self.xgb_es_, verbose=1)
        if save:
          with open(xgb_path, 'wb') as f:
            dill.dump(xgb, f)

      xgb_models_.append(xgb)
    return xgb_models_


  def __hpo_obj(self, models):
    print('EVALUATING NDCG FOR HPO..')
    scores = []
    for model in models:
      preds = model.predict_proba(self.X_val_)
      score = ndcg_score(self.T_val_, preds, k=5, ignore_ties=True)
      scores.append(score)
    return -np.mean(scores)
  

  def __hpo_nn(self, **kwargs):
    nns = self.__fit_nn(save=False, **kwargs)
    return self.__hpo_obj(nns)


  def __nn_hpo_wrapper(self):
    self.T_val_ = self._convert_y_for_testing(self.y_val_)
    res = forest_minimize(self.__hpo_nn, dimensions=nn_space,
                        n_calls=15, n_random_starts=5,
                        acq_func="LCB", random_state=self.random_state)
    best = res.x
    self.nn_bestseen_ = -np.minimum.accumulate(res.func_vals)
    self.nn_best_conf_ = dict(zip(paramnames_nn, best))
    return self.nn_best_conf_


  def __hpo_xgb(self, **kwargs):
    xgbs = self.__fit_xgb(save=False, **kwargs)
    return self.__hpo_obj(xgbs)


  def __xgb_hpo_wrapper(self):
    self.T_val_ = self._convert_y_for_testing(self.y_val_)
    res = gp_minimize(self.__hpo_xgb, dimensions=xgb_space,
                      n_calls=15, n_random_starts=5,
                      acq_func="LCB", random_state=self.random_state)
    best = res.x
    self.xgb_bestseen_ = -np.minimum.accumulate(res.func_vals)
    self.xgb_best_conf_ = dict(zip(paramnames_xgb, best))
    return self.xgb_best_conf_


  def get_prediction_matrix(self, X):
    print("Computing preliminary predictions...")
    predictions = np.empty((X.shape[0], 0))
    for model in self.nn_models_ + self.xgb_models_:
      modelpreds = model.predict_proba(X)
      predictions = np.hstack((predictions, modelpreds))
    return predictions


  def _fit_ensembler(self):
    X_probs = self.get_prediction_matrix(self.X_train_)
    x0 = np.random.uniform(size = self.n_classes_ * len(self.nn_models_ + self.xgb_models_))
    bounds = [(0,1)]*len(x0)   
    res = minimize(self.objective_fun, 
                   x0, 
                   args=(X_probs, self.y_train_, self.n_classes_), 
                   method='L-BFGS-B', 
                   bounds=bounds
                  )
    self.w_ = res.x


  def predict(self, X):
    X = check_array(X)
    pred_probs = self.get_prediction_matrix(X)
    y_pred = np.zeros((X.shape[0], self.n_classes_))
    for i in range(len(self.w_)):
        y_pred[:, i % self.n_classes_] += pred_probs[:, i] * self.w_[i]   
    return y_pred 

    
  def predict_proba(self, X):
    return self.predict(X)
  

  def _convert_y_for_testing(self, y):
    lb = LabelBinarizer()
    lb.fit(range(self.n_classes_))
    T = lb.transform(y)
    return T

  def score(self, X, y):
    y = check_array(y, dtype = np.int_, ensure_2d=False).ravel()
    if type_of_target(y) == 'binary':
      raise SkipTest
    elif y.ndim < 2:
      T = self._convert_y_for_testing(y)
    else:
      T = y
    preds = self.predict(X)
    return ndcg_score(T, preds, k=min(self.n_classes_, 5), ignore_ties=True)

In [0]:
ens = EnsembleClassifier(BOOTSTRAP_ITER=10, load_models=False,
                         save_models=True)

In [47]:
ens.fit(X_train[:, xgb_features], y_train, 
        X_val=X_val[:, xgb_features], 
        y_val=y_val)

FITTING NN ON BOOTSTRAP 0


Train on 49988 samples, validate on 53363 samples
Epoch 1/500
49988/49988 - 6s - loss: 3.3653 - sparse_categorical_accuracy: 0.3998 - sparse_top_k_categorical_accuracy: 0.7721 - val_loss: 1.4209 - val_sparse_categorical_accuracy: 0.6308 - val_sparse_top_k_categorical_accuracy: 0.8137
Epoch 2/500
49988/49988 - 5s - loss: 1.8292 - sparse_categorical_accuracy: 0.4560 - sparse_top_k_categorical_accuracy: 0.8248 - val_loss: 1.3662 - val_sparse_categorical_accuracy: 0.7140 - val_sparse_top_k_categorical_accuracy: 0.8309
Epoch 3/500
49988/49988 - 5s - loss: 1.7776 - sparse_categorical_accuracy: 0.4637 - sparse_top_k_categorical_accuracy: 0.8269 - val_loss: 1.4876 - val_sparse_categorical_accuracy: 0.6011 - val_sparse_top_k_categorical_accuracy: 0.8160
Epoch 4/500
49988/49988 - 5s - loss: 1.7584 - sparse_categorical_accuracy: 0.4649 - sparse_top_k_categorical_accuracy: 0.8326 - val_loss: 1.2687 - val_sparse_categorical_accuracy: 0.6494 - val_sparse_top_k_categorical

EnsembleClassifier(BATCHSIZE=64, BOOTSTRAP_ITER=10, EPOCHS=500, hpo=False,
                   load_models=False, random_state=42, save_models=True)

In [48]:
ens.score(X_val[:, xgb_features], y_val)

Computing preliminary predictions...


0.8351451913844551

In [0]:
cal_ens = CalibratedClassifierCV(ens, method='isotonic', cv='prefit')

In [50]:
cal_ens.fit(X_val[:, xgb_features], y_val)

Computing preliminary predictions...


CalibratedClassifierCV(base_estimator=EnsembleClassifier(BATCHSIZE=64,
                                                         BOOTSTRAP_ITER=10,
                                                         EPOCHS=500, hpo=False,
                                                         load_models=False,
                                                         random_state=42,
                                                         save_models=True),
                       cv='prefit', method='isotonic')

This cell implements the second ensemble model described in the report by extending the previous class and overriding only two methods.

In [0]:
class EnsembleSingleModelsClassifier(EnsembleClassifier):
  @staticmethod
  def objective_fun(w, X, y, n_class):
    w = np.abs(w)
    sol = np.zeros((X.shape[0], n_class))
    for idx in range(len(w)):
      sol += X[:, idx*n_class:(idx+1)*n_class] * w[idx]
    if np.isnan(sol).any():
      return np.inf
    ll = log_loss(y, sol)
    return ll    


  def _fit_ensembler(self):
    X_probs = self.get_prediction_matrix(self.X_train_)
    x0 = np.random.uniform(size = len(self.nn_models_ + self.xgb_models_))
    bounds = [(0,1)]*len(x0)
    cons = ({'type': 'eq', 'fun': lambda w: 1-sum(w)})
    res = minimize(self.objective_fun, 
                    x0, 
                    args=(X_probs, self.y_train_, self.n_classes_), 
                    method='SLSQP', 
                    bounds=bounds
                   )
    self.w_ = res.x 

In [0]:
ens2 = EnsembleSingleModelsClassifier(BOOTSTRAP_ITER=10, load_models=True)

In [53]:
ens2.fit(X_train[:, xgb_features], y_train, 
        X_val=X_val[:, xgb_features], 
        y_val=y_val)

LOADING NN 0...


LOADING NN 1...


LOADING NN 2...


LOADING NN 3...


LOADING NN 4...


LOADING NN 5...


LOADING NN 6...


LOADING NN 7...


LOADING NN 8...


LOADING NN 9...


LOADING XGB 0...


LOADING XGB 1...


LOADING XGB 2...


LOADING XGB 3...


LOADING XGB 4...


LOADING XGB 5...


LOADING XGB 6...


LOADING XGB 7...


LOADING XGB 8...


LOADING XGB 9...


FITTING ENSEMBLER...
Computing preliminary predictions...


EnsembleSingleModelsClassifier(BATCHSIZE=64, BOOTSTRAP_ITER=10, EPOCHS=500,
                               hpo=False, load_models=True, random_state=42,
                               save_models=False)

In [61]:
ens2.score(X_val[:, xgb_features], y_val)

Computing preliminary predictions...


0.7251559637858063

# RERUN WITH HPO

---



In [0]:
hpo_ens = EnsembleSingleModelsClassifier(BOOTSTRAP_ITER=2, load_models=False, 
                                         save_models=False, hpo=True)

In [0]:
hpo_ens.fit(X_train[:, xgb_features], y_train, 
        X_val=X_val[:, xgb_features], 
        y_val=y_val)

ENTERING HPO...
FITTING XGB ON BOOTSTRAP 0


[0]	validation_0-merror:0.367389	validation_0-mlogloss:2.14499
Multiple eval metrics have been passed: 'validation_0-mlogloss' will be used for early stopping.

Will train until validation_0-mlogloss hasn't improved in 50 rounds.
[1]	validation_0-merror:0.357963	validation_0-mlogloss:1.95465
[2]	validation_0-merror:0.355583	validation_0-mlogloss:1.82289
[3]	validation_0-merror:0.356652	validation_0-mlogloss:1.72164
[4]	validation_0-merror:0.351667	validation_0-mlogloss:1.63423
[5]	validation_0-merror:0.34998	validation_0-mlogloss:1.56494
[6]	validation_0-merror:0.346851	validation_0-mlogloss:1.50447
[7]	validation_0-merror:0.34612	validation_0-mlogloss:1.45307
[8]	validation_0-merror:0.342597	validation_0-mlogloss:1.40967
[9]	validation_0-merror:0.342316	validation_0-mlogloss:1.37299
[10]	validation_0-merror:0.341154	validation_0-mlogloss:1.33904
[11]	validation_0-merror:0.340048	validation_0-mlogloss:1.31204
[12]	validation_0-merror:0.33877

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


FITTING XGB ON BOOTSTRAP 0


[0]	validation_0-merror:0.378502	validation_0-mlogloss:2.14677
Multiple eval metrics have been passed: 'validation_0-mlogloss' will be used for early stopping.

Will train until validation_0-mlogloss hasn't improved in 50 rounds.
[1]	validation_0-merror:0.37005	validation_0-mlogloss:1.96046
[2]	validation_0-merror:0.370125	validation_0-mlogloss:1.8297
[3]	validation_0-merror:0.369526	validation_0-mlogloss:1.72876
[4]	validation_0-merror:0.367764	validation_0-mlogloss:1.64408
[5]	validation_0-merror:0.364054	validation_0-mlogloss:1.5716
[6]	validation_0-merror:0.358994	validation_0-mlogloss:1.51075
[7]	validation_0-merror:0.355452	validation_0-mlogloss:1.45974
[8]	validation_0-merror:0.350767	validation_0-mlogloss:1.41442
[9]	validation_0-merror:0.350486	validation_0-mlogloss:1.37584
[10]	validation_0-merror:0.349531	validation_0-mlogloss:1.34178
[11]	validation_0-merror:0.351161	validation_0-mlogloss:1.31219
[12]	validation_0-merror:0.351498	validation_0-ml

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


FITTING XGB ON BOOTSTRAP 0


[0]	validation_0-merror:0.382718	validation_0-mlogloss:2.15031
Multiple eval metrics have been passed: 'validation_0-mlogloss' will be used for early stopping.

Will train until validation_0-mlogloss hasn't improved in 50 rounds.
[1]	validation_0-merror:0.36426	validation_0-mlogloss:1.96183
[2]	validation_0-merror:0.366096	validation_0-mlogloss:1.83089
[3]	validation_0-merror:0.362929	validation_0-mlogloss:1.72849
[4]	validation_0-merror:0.367464	validation_0-mlogloss:1.64813
[5]	validation_0-merror:0.362142	validation_0-mlogloss:1.57786
[6]	validation_0-merror:0.363398	validation_0-mlogloss:1.52229
[7]	validation_0-merror:0.359819	validation_0-mlogloss:1.47106
[8]	validation_0-merror:0.35935	validation_0-mlogloss:1.42446
[9]	validation_0-merror:0.356708	validation_0-mlogloss:1.38745
[10]	validation_0-merror:0.357551	validation_0-mlogloss:1.35801
[11]	validation_0-merror:0.356089	validation_0-mlogloss:1.32511
[12]	validation_0-merror:0.353597	validation_0-m

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


FITTING XGB ON BOOTSTRAP 0


[0]	validation_0-merror:0.356239	validation_0-mlogloss:2.14068
Multiple eval metrics have been passed: 'validation_0-mlogloss' will be used for early stopping.

Will train until validation_0-mlogloss hasn't improved in 50 rounds.
[1]	validation_0-merror:0.357645	validation_0-mlogloss:1.95538
[2]	validation_0-merror:0.356445	validation_0-mlogloss:1.82163
[3]	validation_0-merror:0.352698	validation_0-mlogloss:1.71435
[4]	validation_0-merror:0.349924	validation_0-mlogloss:1.62852
[5]	validation_0-merror:0.349231	validation_0-mlogloss:1.5584
[6]	validation_0-merror:0.349381	validation_0-mlogloss:1.49863
[7]	validation_0-merror:0.350261	validation_0-mlogloss:1.45062
[8]	validation_0-merror:0.350842	validation_0-mlogloss:1.40753
[9]	validation_0-merror:0.349905	validation_0-mlogloss:1.36887
[10]	validation_0-merror:0.348425	validation_0-mlogloss:1.33421
[11]	validation_0-merror:0.345726	validation_0-mlogloss:1.30099
[12]	validation_0-merror:0.34419	validation_0-m

In [0]:
hpo_ens.w_